In [1]:
#Importing Libraries
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
import scipy.special
import statsmodels.api as sm
import dask.dataframe as dd
from dask_ml.preprocessing import QuantileTransformer
!pip install graphviz
import graphviz
from dask_ml.preprocessing import PolynomialFeatures
from dask_ml.datasets import make_regression


Defaulting to user installation because normal site-packages is not writeable


In [2]:
#Importing dataset and removing rows with store number = 1020
stime = time.time()

print("Started at, ", stime, "Reading file")
#filters = pd.read_csv("/home/vsoni/AAP/filters/pi/filters.csv", sep='|')
filters = dd.read_csv("/home/vsoni/AAP/filters/pi/filters.csv", sep='|',dtype={'OTHER_UNIT_PLS_LOST_SALES_CY': 'float64',
       'OTHER_UNIT_PLS_LOST_SALES_PPY': 'float64',
       'OTHER_UNIT_PLS_LOST_SALES_PY': 'float64',
       'PLATFORM_CLUSTER_NAME': 'float64'})

filters= filters[filters['STORE_NUMBER']!=1020]
filters

Started at,  1649101707.9256468 Reading file


,SKU_NUMBER,STORE_NUMBER,PART_TYPE,PLATFORM_CLUSTER_NAME,APPLICATION_COUNT,POP_EST_CY,POP_DENSITY_CY,PCT_WHITE,AGE,PCT_COLLEGE,PCT_BLUE_COLLAR,MEDIAN_HOUSEHOLD_INCOME,ESTABLISHMENTS,SKU_EXISTENCE_PY,SKU_EXISTENCE_CY,SKU_STORE_PDQ_PY,SKU_STORE_PDQ_CY,TOTAL_VIO_PY,TOTAL_VIO_CY,UNADJUSTED_TOTAL_VIO_PY,UNADJUSTED_TOTAL_VIO_CY,LOST_QTY_PY,LOST_QTY_CY,SS_SALES_PY,SS_SALES_CY,AVG_CLUSTER_UNIT_SALES_PY,AVG_CLUSTER_UNIT_SALES_CY,AVG_CLUSTER_LOST_SALES_PY,AVG_CLUSTER_LOST_SALES_CY,ROAD_QUALITY_INDEX,VIO_COMPARED_TO_CLUSTER_PY,VIO_COMPARED_TO_CLUSTER_CY,ADJUSTED_AVG_CLUSTER_SALES_PY,ADJUSTED_AVG_CLUSTER_SALES_CY,QTY_SOLD_PPY,QTY_SOLD_PY,QTY_SOLD_CY,AVG_CLUSTER_TOTAL_SALES_PY,AVG_CLUSTER_TOTAL_SALES_CY,SALES_SIGNAL_PY,SALES_SIGNAL_CY,FAILURE_SALES_PY,FAILURE_SALES_CY,LIFECYCLE_PY,LIFECYCLE_CY,ADJUSTED_LIFECYCLE_PY,ADJUSTED_LIFECYCLE_CY,PCT_OF_LIFECYCLE_REMAINING,LIFECYCLE_PRE_PEAK_POST,ADJ_AVG_CLUSTER_LOST_SALES_PY,ADJ_AVG_CLUSTER_LOST_SALES_CY,ADJ_AVG_CLUSTER_TOTAL_SALES_PY,ADJ_AVG_CLUSTER_TOTAL_SALES_CY,SOLD_SINCE_MAXI,cy_unit_sales,py_unit_sales,ppy_unit_sales,cy_gross_sales,py_gross_sales,ppy_gross_sales,cy_sales_cost,py_sales_cost,ppy_sales_cost,other_unit_sales,other_gross_sales,other_sales_cost,PROJECTED_GROWTH_PCT_PY,PROJECTED_GROWTH_PCT_CY,OTHER_UNIT_PLS_LOST_SALES_PPY,OTHER_UNIT_PLS_LOST_SALES_PY,OTHER_UNIT_PLS_LOST_SALES_CY,LOOKUP_CNT_PY,LOOKUP_CNT_CY,WEIGHTED_LOOKUP_CNT_PY,WEIGHTED_LOOKUP_CNT_CY,CY_PERIODS_IN_STOCK,WAIT_TIME_BUCKET,MPOG_ID,DATA_CLASSIFICATION,cy_qty_sold_transfer,cy_qty_sold_on_hand,py_qty_sold_transfer,py_qty_sold_on_hand,ppy_qty_sold_transfer,ppy_qty_sold_on_hand,ON_HAND,fy_ts_forecast,cy_ts_forecast
npartitions=287,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,object,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,object,float64,float64,float64,float64,float64,float64,object,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
filters.head()

,SKU_NUMBER,STORE_NUMBER,PART_TYPE,PLATFORM_CLUSTER_NAME,APPLICATION_COUNT,POP_EST_CY,POP_DENSITY_CY,PCT_WHITE,AGE,PCT_COLLEGE,...,DATA_CLASSIFICATION,cy_qty_sold_transfer,cy_qty_sold_on_hand,py_qty_sold_transfer,py_qty_sold_on_hand,ppy_qty_sold_transfer,ppy_qty_sold_on_hand,ON_HAND,fy_ts_forecast,cy_ts_forecast
0,5792231,2127,AIR FILTER,55.0,306,13297.00,0.003481,0.62,32.44,0.16,...,NON PR WITH EPC,0.0,2.0,0.0,3.0,0.0,4.0,Y,4629.0,5068.0
1,10162622,4810,AIR FILTER,17.0,13,6461.82,0.002970,0.34,31.92,0.15,...,NON PR WITH EPC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,75.0
2,5792231,4165,AIR FILTER,70.0,306,13524.37,0.007584,0.02,36.63,0.18,...,NON PR WITH EPC,0.0,2.0,0.0,0.0,0.0,1.0,Y,4629.0,5068.0
3,10162622,1039,AIR FILTER,52.0,13,5981.02,0.004431,0.12,34.64,0.15,...,NON PR WITH EPC,0.0,0.0,0.0,0.0,0.0,0.0,NaN,52.0,75.0
4,5792231,4500,AIR FILTER,66.0,306,16990.95,0.003805,0.27,34.05,0.13,...,NON PR WITH EPC,0.0,5.0,0.0,4.0,0.0,2.0,N,4629.0,5068.0


In [4]:
#Creating a list of features to be dropped. Features with 'cy' barring 'QTY_SOLD_CY', 'PROJECTED_GROWTH_PCT_CY' and POP_DENSITY_CY
drop_columns = []
for i in filters.columns:
    if ('_cy' in i.lower() or 'cy_' in i.lower() or 'lost' in i.lower()) and i!='QTY_SOLD_CY' and i!='PROJECTED_GROWTH_PCT_CY' and i!='POP_DENSITY_CY':
        drop_columns.append(i)
    else:
        continue
drop_columns

['POP_EST_CY',
 'SKU_EXISTENCE_CY',
 'SKU_STORE_PDQ_CY',
 'TOTAL_VIO_CY',
 'UNADJUSTED_TOTAL_VIO_CY',
 'LOST_QTY_PY',
 'LOST_QTY_CY',
 'SS_SALES_CY',
 'AVG_CLUSTER_UNIT_SALES_CY',
 'AVG_CLUSTER_LOST_SALES_PY',
 'AVG_CLUSTER_LOST_SALES_CY',
 'VIO_COMPARED_TO_CLUSTER_CY',
 'ADJUSTED_AVG_CLUSTER_SALES_CY',
 'AVG_CLUSTER_TOTAL_SALES_CY',
 'SALES_SIGNAL_CY',
 'FAILURE_SALES_CY',
 'LIFECYCLE_CY',
 'ADJUSTED_LIFECYCLE_CY',
 'ADJ_AVG_CLUSTER_LOST_SALES_PY',
 'ADJ_AVG_CLUSTER_LOST_SALES_CY',
 'ADJ_AVG_CLUSTER_TOTAL_SALES_CY',
 'cy_unit_sales',
 'cy_gross_sales',
 'cy_sales_cost',
 'OTHER_UNIT_PLS_LOST_SALES_PPY',
 'OTHER_UNIT_PLS_LOST_SALES_PY',
 'OTHER_UNIT_PLS_LOST_SALES_CY',
 'LOOKUP_CNT_CY',
 'WEIGHTED_LOOKUP_CNT_CY',
 'CY_PERIODS_IN_STOCK',
 'cy_qty_sold_transfer',
 'cy_qty_sold_on_hand',
 'cy_ts_forecast']

In [6]:
#List of all features
filters.columns

Index(['SKU_NUMBER', 'STORE_NUMBER', 'PART_TYPE', 'PLATFORM_CLUSTER_NAME',
       'APPLICATION_COUNT', 'POP_EST_CY', 'POP_DENSITY_CY', 'PCT_WHITE', 'AGE',
       'PCT_COLLEGE', 'PCT_BLUE_COLLAR', 'MEDIAN_HOUSEHOLD_INCOME',
       'ESTABLISHMENTS', 'SKU_EXISTENCE_PY', 'SKU_EXISTENCE_CY',
       'SKU_STORE_PDQ_PY', 'SKU_STORE_PDQ_CY', 'TOTAL_VIO_PY', 'TOTAL_VIO_CY',
       'UNADJUSTED_TOTAL_VIO_PY', 'UNADJUSTED_TOTAL_VIO_CY', 'LOST_QTY_PY',
       'LOST_QTY_CY', 'SS_SALES_PY', 'SS_SALES_CY',
       'AVG_CLUSTER_UNIT_SALES_PY', 'AVG_CLUSTER_UNIT_SALES_CY',
       'AVG_CLUSTER_LOST_SALES_PY', 'AVG_CLUSTER_LOST_SALES_CY',
       'ROAD_QUALITY_INDEX', 'VIO_COMPARED_TO_CLUSTER_PY',
       'VIO_COMPARED_TO_CLUSTER_CY', 'ADJUSTED_AVG_CLUSTER_SALES_PY',
       'ADJUSTED_AVG_CLUSTER_SALES_CY', 'QTY_SOLD_PPY', 'QTY_SOLD_PY',
       'QTY_SOLD_CY', 'AVG_CLUSTER_TOTAL_SALES_PY',
       'AVG_CLUSTER_TOTAL_SALES_CY', 'SALES_SIGNAL_PY', 'SALES_SIGNAL_CY',
       'FAILURE_SALES_PY', 'FAILURE_SALES_CY', '

In [7]:
#Adding additional features to be dropped in the list and dropping those features from the dataset
drop_columns.extend(['PLATFORM_CLUSTER_NAME','UNADJUSTED_TOTAL_VIO_PY', 'UNADJUSTED_TOTAL_VIO_CY', 'SS_SALES_PY', 'FAILURE_SALES_PY','LIFECYCLE_PY', 'AVG_CLUSTER_LOST_SALES_PY', 'other_unit_sales',
       'other_gross_sales', 'AVG_CLUSTER_TOTAL_SALES_PY', 'other_sales_cost', 'LOOKUP_CNT_PY', 'WEIGHTED_LOOKUP_CNT_PY', 'py_qty_sold_transfer', 'py_qty_sold_on_hand',
       'ppy_qty_sold_transfer', 'ppy_qty_sold_on_hand', 'ON_HAND', 'fy_ts_forecast', 'WAIT_TIME_BUCKET'])
filters = filters.drop(drop_columns,axis=1)
filters.info()


<class 'dask.dataframe.core.DataFrame'>
Columns: 37 entries, SKU_NUMBER to DATA_CLASSIFICATION
dtypes: object(2), float64(27), int64(8)

In [8]:
#Imputation of 'PART_TYPE', 'ROAD_QUALITY_INDEX', 'PROJECTED_GROWTH_PCT_CY' and 'PROJECTED_GROWTH_PCT_PY'
filters['PART_TYPE'] = filters.PART_TYPE.fillna('Miscellaneous')
mean_rqi = filters[['STORE_NUMBER','ROAD_QUALITY_INDEX']].drop_duplicates()['ROAD_QUALITY_INDEX'].mean().compute()
filters.ROAD_QUALITY_INDEX = filters.ROAD_QUALITY_INDEX.fillna(mean_rqi)
filters.PROJECTED_GROWTH_PCT_CY = filters.PROJECTED_GROWTH_PCT_CY.fillna('mean')
filters.PROJECTED_GROWTH_PCT_PY = filters.PROJECTED_GROWTH_PCT_PY.fillna('mean')

/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (2,85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=

In [9]:
#Dropping additional features and checking for null values in remaining columns
filters = filters.drop(['py_unit_sales','ppy_unit_sales','ppy_gross_sales','py_sales_cost','ppy_sales_cost','TOTAL_VIO_PY'],axis=1)
filters.isna().sum().compute()

/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (2,85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(


SKU_NUMBER                              0
STORE_NUMBER                            0
PART_TYPE                               0
APPLICATION_COUNT                       0
POP_DENSITY_CY                          0
PCT_WHITE                               0
AGE                                     0
PCT_COLLEGE                             0
PCT_BLUE_COLLAR                         0
MEDIAN_HOUSEHOLD_INCOME                 0
ESTABLISHMENTS                    3626100
SKU_EXISTENCE_PY                        0
SKU_STORE_PDQ_PY                        0
AVG_CLUSTER_UNIT_SALES_PY               0
ROAD_QUALITY_INDEX                      0
VIO_COMPARED_TO_CLUSTER_PY              0
ADJUSTED_AVG_CLUSTER_SALES_PY           0
QTY_SOLD_PPY                            0
QTY_SOLD_PY                             0
QTY_SOLD_CY                             0
SALES_SIGNAL_PY                         0
ADJUSTED_LIFECYCLE_PY                   0
PCT_OF_LIFECYCLE_REMAINING              0
LIFECYCLE_PRE_PEAK_POST           

In [10]:
#Creating a list of SKUs with py_gross_sales != 0 across all stores
sku_list = set(filters[filters['py_gross_sales']!=0]['SKU_NUMBER'])
sku_list

/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (2,85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(


{11591680,
 22880268,
 22880269,
 22880270,
 5791758,
 22880271,
 22880273,
 5791766,
 11903003,
 5791782,
 12083262,
 12083264,
 22880321,
 22880322,
 22880323,
 22880324,
 11395141,
 11395142,
 22880327,
 11395143,
 11395145,
 22880330,
 11395146,
 11395147,
 22880331,
 11395150,
 11395151,
 22880332,
 11395152,
 11395154,
 11395155,
 22880340,
 11395157,
 22880336,
 11395159,
 22880344,
 22880343,
 22880346,
 22880347,
 11395162,
 22880349,
 22880348,
 11395163,
 11395166,
 11395169,
 11395165,
 11395171,
 22880356,
 11395168,
 22880351,
 5791847,
 11395172,
 22880366,
 5791855,
 5791863,
 10051708,
 22880381,
 5791871,
 10838144,
 10838145,
 10051713,
 22880384,
 22880386,
 10838148,
 10051711,
 5790069,
 10838152,
 10051721,
 10838154,
 11591818,
 11935884,
 11935883,
 11935886,
 11935887,
 11264144,
 10838160,
 11591822,
 11395219,
 11395220,
 11395221,
 10838165,
 11395224,
 11395225,
 11395226,
 11395227,
 5791898,
 5791901,
 22880415,
 11395231,
 11935905,
 11395237,
 11395238

In [11]:
#all_skus = set(filters['SKU_NUMBER'])
#len(all_skus)

In [12]:
#Removing all SKUs with py_gross_sales = 0 across all stores from the dataset
filters = filters[filters['SKU_NUMBER'].isin(sku_list)]

In [13]:
#Replacing negative values in QTY_SOLD_CY with 0
filters[filters['QTY_SOLD_CY']<0]['QTY_SOLD_CY'] = 0

In [14]:
#Fixing the datatypes
filters.PROJECTED_GROWTH_PCT_PY = filters.PROJECTED_GROWTH_PCT_PY.astype(float)
filters.PROJECTED_GROWTH_PCT_CY = filters.PROJECTED_GROWTH_PCT_CY.astype(float)

In [15]:
#Splitting dataset into predictors and target variable
y = filters['QTY_SOLD_CY']
filters = filters.drop(['QTY_SOLD_CY','SKU_NUMBER','STORE_NUMBER'],axis=1)
x = filters

In [17]:
#Converting dask dataframe to pandas dataframe
x = x.compute()
y = y.compute()
print(type(x))
print(type(y))

/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (2,85) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/apps/cent7/jupyterhub/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [18]:
#Imputing ESTABLISHMENTS using POP_DENSITY_CY
x['POP_DENSITY_CY_QT'] = pd.qcut(x['POP_DENSITY_CY'], q=4, labels=['a','b','c','d'])
x["ESTABLISHMENTS"]= x.groupby('POP_DENSITY_CY_QT')['ESTABLISHMENTS'].transform(lambda x: x.mean())
x[x["ESTABLISHMENTS"].isnull()]

,PART_TYPE,APPLICATION_COUNT,POP_DENSITY_CY,PCT_WHITE,AGE,PCT_COLLEGE,PCT_BLUE_COLLAR,MEDIAN_HOUSEHOLD_INCOME,ESTABLISHMENTS,SKU_EXISTENCE_PY,...,PCT_OF_LIFECYCLE_REMAINING,LIFECYCLE_PRE_PEAK_POST,ADJ_AVG_CLUSTER_TOTAL_SALES_PY,SOLD_SINCE_MAXI,py_gross_sales,PROJECTED_GROWTH_PCT_PY,PROJECTED_GROWTH_PCT_CY,MPOG_ID,DATA_CLASSIFICATION,POP_DENSITY_CY_QT


In [19]:
#Dropping POP_DENSITY_CY and POP_DENSITY_CY_QT and checking for null values in features
x = x.drop(['POP_DENSITY_CY_QT','POP_DENSITY_CY'],axis=1)
x.isna().sum()

PART_TYPE                         0
APPLICATION_COUNT                 0
PCT_WHITE                         0
AGE                               0
PCT_COLLEGE                       0
PCT_BLUE_COLLAR                   0
MEDIAN_HOUSEHOLD_INCOME           0
ESTABLISHMENTS                    0
SKU_EXISTENCE_PY                  0
SKU_STORE_PDQ_PY                  0
AVG_CLUSTER_UNIT_SALES_PY         0
ROAD_QUALITY_INDEX                0
VIO_COMPARED_TO_CLUSTER_PY        0
ADJUSTED_AVG_CLUSTER_SALES_PY     0
QTY_SOLD_PPY                      0
QTY_SOLD_PY                       0
SALES_SIGNAL_PY                   0
ADJUSTED_LIFECYCLE_PY             0
PCT_OF_LIFECYCLE_REMAINING        0
LIFECYCLE_PRE_PEAK_POST           0
ADJ_AVG_CLUSTER_TOTAL_SALES_PY    0
SOLD_SINCE_MAXI                   0
py_gross_sales                    0
PROJECTED_GROWTH_PCT_PY           0
PROJECTED_GROWTH_PCT_CY           0
MPOG_ID                           0
DATA_CLASSIFICATION               0
dtype: int64

In [20]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18053828 entries, 0 to 74700
Data columns (total 27 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   PART_TYPE                       object 
 1   APPLICATION_COUNT               int64  
 2   PCT_WHITE                       float64
 3   AGE                             float64
 4   PCT_COLLEGE                     float64
 5   PCT_BLUE_COLLAR                 float64
 6   MEDIAN_HOUSEHOLD_INCOME         float64
 7   ESTABLISHMENTS                  float64
 8   SKU_EXISTENCE_PY                int64  
 9   SKU_STORE_PDQ_PY                int64  
 10  AVG_CLUSTER_UNIT_SALES_PY       float64
 11  ROAD_QUALITY_INDEX              float64
 12  VIO_COMPARED_TO_CLUSTER_PY      float64
 13  ADJUSTED_AVG_CLUSTER_SALES_PY   float64
 14  QTY_SOLD_PPY                    float64
 15  QTY_SOLD_PY                     float64
 16  SALES_SIGNAL_PY                 float64
 17  ADJUSTED_LIFECYCLE_PY       

In [21]:
#Fixing ESTABLISHMENTS datatype
x.ESTABLISHMENTS = x.ESTABLISHMENTS.astype(float)
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18053828 entries, 0 to 74700
Data columns (total 27 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   PART_TYPE                       object 
 1   APPLICATION_COUNT               int64  
 2   PCT_WHITE                       float64
 3   AGE                             float64
 4   PCT_COLLEGE                     float64
 5   PCT_BLUE_COLLAR                 float64
 6   MEDIAN_HOUSEHOLD_INCOME         float64
 7   ESTABLISHMENTS                  float64
 8   SKU_EXISTENCE_PY                int64  
 9   SKU_STORE_PDQ_PY                int64  
 10  AVG_CLUSTER_UNIT_SALES_PY       float64
 11  ROAD_QUALITY_INDEX              float64
 12  VIO_COMPARED_TO_CLUSTER_PY      float64
 13  ADJUSTED_AVG_CLUSTER_SALES_PY   float64
 14  QTY_SOLD_PPY                    float64
 15  QTY_SOLD_PY                     float64
 16  SALES_SIGNAL_PY                 float64
 17  ADJUSTED_LIFECYCLE_PY       

In [22]:
#Creating a list of categorical and numerical Columns
num_cols = []
cat_cols = []
for i in x.columns:
    if x[i].dtype=='object':
        cat_cols.append(i)
    else:
        num_cols.append(i)
num_cols

['APPLICATION_COUNT',
 'PCT_WHITE',
 'AGE',
 'PCT_COLLEGE',
 'PCT_BLUE_COLLAR',
 'MEDIAN_HOUSEHOLD_INCOME',
 'ESTABLISHMENTS',
 'SKU_EXISTENCE_PY',
 'SKU_STORE_PDQ_PY',
 'AVG_CLUSTER_UNIT_SALES_PY',
 'ROAD_QUALITY_INDEX',
 'VIO_COMPARED_TO_CLUSTER_PY',
 'ADJUSTED_AVG_CLUSTER_SALES_PY',
 'QTY_SOLD_PPY',
 'QTY_SOLD_PY',
 'SALES_SIGNAL_PY',
 'ADJUSTED_LIFECYCLE_PY',
 'PCT_OF_LIFECYCLE_REMAINING',
 'LIFECYCLE_PRE_PEAK_POST',
 'ADJ_AVG_CLUSTER_TOTAL_SALES_PY',
 'SOLD_SINCE_MAXI',
 'py_gross_sales',
 'PROJECTED_GROWTH_PCT_PY',
 'PROJECTED_GROWTH_PCT_CY',
 'MPOG_ID']

In [23]:
#List of all columns
all_cols = ['APPLICATION_COUNT',
 'PCT_WHITE',
 'AGE',
 'PCT_COLLEGE',
 'PCT_BLUE_COLLAR',
 'MEDIAN_HOUSEHOLD_INCOME',
 'ESTABLISHMENTS',
 'SKU_EXISTENCE_PY',
 'SKU_STORE_PDQ_PY',
 'AVG_CLUSTER_UNIT_SALES_PY',
 'ROAD_QUALITY_INDEX',
 'VIO_COMPARED_TO_CLUSTER_PY',
 'ADJUSTED_AVG_CLUSTER_SALES_PY',
 'QTY_SOLD_PPY',
 'QTY_SOLD_PY',
 'SALES_SIGNAL_PY',
 'ADJUSTED_LIFECYCLE_PY',
 'PCT_OF_LIFECYCLE_REMAINING',
 'LIFECYCLE_PRE_PEAK_POST',
 'ADJ_AVG_CLUSTER_TOTAL_SALES_PY',
 'SOLD_SINCE_MAXI',
 'py_gross_sales',
 'PROJECTED_GROWTH_PCT_PY',
 'PROJECTED_GROWTH_PCT_CY',
 'MPOG_ID','PART_TYPE', 'DATA_CLASSIFICATION']

In [24]:
#Standardization of numerical features
ct = ColumnTransformer([
        ('scaler', StandardScaler(), ['APPLICATION_COUNT',
 'PCT_WHITE',
 'AGE',
 'PCT_COLLEGE',
 'PCT_BLUE_COLLAR',
 'MEDIAN_HOUSEHOLD_INCOME',
'ESTABLISHMENTS',
 'SKU_EXISTENCE_PY',
 'SKU_STORE_PDQ_PY',
    'SALES_SIGNAL_PY',
 'AVG_CLUSTER_UNIT_SALES_PY',
    'ROAD_QUALITY_INDEX',
 'VIO_COMPARED_TO_CLUSTER_PY',
 'ADJUSTED_AVG_CLUSTER_SALES_PY',
 'QTY_SOLD_PPY',
 'QTY_SOLD_PY',
 'ADJUSTED_LIFECYCLE_PY',
 'PCT_OF_LIFECYCLE_REMAINING',
 'LIFECYCLE_PRE_PEAK_POST',
 'SOLD_SINCE_MAXI',
 'PROJECTED_GROWTH_PCT_PY',
 'PROJECTED_GROWTH_PCT_CY',
    'MPOG_ID','py_gross_sales'])
    ], remainder='passthrough')

x = pd.DataFrame(ct.fit_transform(x), columns=all_cols)

In [25]:
#Dropping certain features which are not important as per discussion and fixing feature datatypes
x.drop(['ADJUSTED_LIFECYCLE_PY','LIFECYCLE_PRE_PEAK_POST','MPOG_ID','PART_TYPE','DATA_CLASSIFICATION','SALES_SIGNAL_PY','ADJ_AVG_CLUSTER_TOTAL_SALES_PY','py_gross_sales'], axis=1, inplace=True)
num_cols = ['APPLICATION_COUNT',
 'PCT_WHITE',
 'AGE',
 'PCT_COLLEGE',
 'PCT_BLUE_COLLAR',
 'MEDIAN_HOUSEHOLD_INCOME',
 'ESTABLISHMENTS',
 'SKU_EXISTENCE_PY',
 'SKU_STORE_PDQ_PY',
 'AVG_CLUSTER_UNIT_SALES_PY',
 'ROAD_QUALITY_INDEX',
 'VIO_COMPARED_TO_CLUSTER_PY',
 'ADJUSTED_AVG_CLUSTER_SALES_PY',
 'QTY_SOLD_PPY',
 'QTY_SOLD_PY',
 'PCT_OF_LIFECYCLE_REMAINING',
 'SOLD_SINCE_MAXI',
 'PROJECTED_GROWTH_PCT_PY',
 'PROJECTED_GROWTH_PCT_CY']
for i in x.columns:
    if x[i].dtype=='object':
        cat_cols.append(i)
    else:
        num_cols.append(i)
for i in num_cols:
    x[i] = x[i].astype('float')
x.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18053828 entries, 0 to 18053827
Data columns (total 19 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   APPLICATION_COUNT              float64
 1   PCT_WHITE                      float64
 2   AGE                            float64
 3   PCT_COLLEGE                    float64
 4   PCT_BLUE_COLLAR                float64
 5   MEDIAN_HOUSEHOLD_INCOME        float64
 6   ESTABLISHMENTS                 float64
 7   SKU_EXISTENCE_PY               float64
 8   SKU_STORE_PDQ_PY               float64
 9   AVG_CLUSTER_UNIT_SALES_PY      float64
 10  ROAD_QUALITY_INDEX             float64
 11  VIO_COMPARED_TO_CLUSTER_PY     float64
 12  ADJUSTED_AVG_CLUSTER_SALES_PY  float64
 13  QTY_SOLD_PPY                   float64
 14  QTY_SOLD_PY                    float64
 15  PCT_OF_LIFECYCLE_REMAINING     float64
 16  SOLD_SINCE_MAXI                float64
 17  PROJECTED_GROWTH_PCT_PY        float64
 18  

In [26]:
#Creating selective interactions with QTY_SOLD_PY and QTY_SOLD_PPY
allcols = x.columns
for i in allcols:
    if x[i].dtype == 'float64':
        x['QTY_SOLD_PY & '+i] = x['QTY_SOLD_PY']*x[i]
        x['QTY_SOLD_PPY & '+i] = x['QTY_SOLD_PPY']*x[i]
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18053828 entries, 0 to 18053827
Data columns (total 57 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   APPLICATION_COUNT                             float64
 1   PCT_WHITE                                     float64
 2   AGE                                           float64
 3   PCT_COLLEGE                                   float64
 4   PCT_BLUE_COLLAR                               float64
 5   MEDIAN_HOUSEHOLD_INCOME                       float64
 6   ESTABLISHMENTS                                float64
 7   SKU_EXISTENCE_PY                              float64
 8   SKU_STORE_PDQ_PY                              float64
 9   AVG_CLUSTER_UNIT_SALES_PY                     float64
 10  ROAD_QUALITY_INDEX                            float64
 11  VIO_COMPARED_TO_CLUSTER_PY                    float64
 12  ADJUSTED_AVG_CLUSTER_SALES_PY                 float64


In [27]:
#Dropping certain columns that were not significant in the model and making sure that there are no interactions across different years (Eg. 'PY' with 'PPY')
all_cols = x.columns
cols_dropped = ['QTY_SOLD_PPY & PCT_OF_LIFECYCLE_REMAINING','QTY_SOLD_PY & MEDIAN_HOUSEHOLD_INCOME']
for i in all_cols:
    if (('_PY' in i) and ('_PPY' in i)) or (('_CY' in i) and ('_PY' in i)) or (('_CY' in i) and ('_PPY' in i)):
        cols_dropped.append(i)
x.drop(cols_dropped, axis=1, inplace=True)

In [28]:
#Creating train and validation sets
x, x_val, y, y_val = train_test_split(x, y, test_size=0.3, random_state=0)

In [29]:
y = y.tolist()
y_val = y_val.tolist()

In [31]:
#Running OLS model
#x = sm.add_constant(x)
olsmodel = sm.OLS(y,x)
results = olsmodel.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.751
Model:                            OLS   Adj. R-squared (uncentered):              0.751
Method:                 Least Squares   F-statistic:                          8.464e+05
Date:                Mon, 04 Apr 2022   Prob (F-statistic):                        0.00
Time:                        16:08:00   Log-Likelihood:                     -1.9044e+07
No. Observations:            12637679   AIC:                                  3.809e+07
Df Residuals:                12637634   BIC:                                  3.809e+07
Df Model:                          45                                                  
Covariance Type:            nonrobust                                                  
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------

In [32]:
#MSE_Residual
results.mse_resid

1.1924776989085504

In [33]:
#Predictions for validation set
y_preds = results.predict(x_val)

In [34]:
#Validation residuals
residuals = y_val-y_preds

In [35]:
#Validation MSE
MSE = np.sum(residuals**2)/len(y_val)
MSE

2.862648460121132

In [36]:
#Mean absolute error on validation set
from statsmodels.tools.eval_measures import meanabs

mae = meanabs(y_val,y_preds)
mae

0.22787186743161392